In [21]:
!git clone https://github.com/YSdoubleM/Yooninahong.git

Cloning into 'Yooninahong'...
remote: Enumerating objects: 8054, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 8054 (delta 3), reused 12 (delta 3), pack-reused 8040
Receiving objects: 100% (8054/8054), 271.45 MiB | 33.58 MiB/s, done.
Resolving deltas: 100% (152/152), done.
Checking out files: 100% (10459/10459), done.


In [22]:
def translate_coord(image,bbs):
  x_c = float(bbs[0])/image.shape[1]
  y_c = float(bbs[1])/image.shape[0]
  h = float(bbs[2])/image.shape[0]
  w = float(bbs[3])/image.shape[1]
  x_c = round(x_c,6)
  y_c = round(y_c,6)
  h = round(h,6)
  w = round(w,6)
  return x_c,y_c,h,w

In [23]:
def augmentation_angle(image_list,path,angle):
  sometimes = lambda aug: iaa.Sometimes(0.5, aug)
  i=0
  for file in image_list:
    image = imageio.imread(path+"/images/train/"+file)
    labelfile = file.replace('.jpg','')
    labelfile = labelfile.replace('.png','')
    labelfile = labelfile+'.txt'
    try:
      f = open(path+'/labels/train/'+labelfile)
    except:
      continue
    boundingboxes = f.readlines()
    new_bb = []
    for box in boundingboxes:
      info_list = box.split()

      class_name = info_list[0]
      x_c = float(info_list[1])*image.shape[1]
      y_c = float(info_list[2])*image.shape[0]
      h = float(info_list[3])*image.shape[0]
      w = float(info_list[4])*image.shape[1]
      x_min = x_c - w / 2
      y_min = y_c - h / 2
      x_max = x_c + w / 2
      y_max = y_c + h / 2

      new_bb.append(ia.BoundingBox(x_min, y_min, x_max, y_max))
    
    bbs = ia.BoundingBoxesOnImage(new_bb, shape = image.shape)

    seq = iaa.Sequential([
    sometimes(iaa.CoarseDropout((0.0, 0.05), size_percent=(0.2, 0.4))),
    sometimes(iaa.Add((-40, 40))),
    ], random_order=True)  # apply augmenters in random order 

    image_aug,bbs_aug = seq(image=image, bounding_boxes = bbs)
    if image_aug.shape[0]<10:
      continue

    bbs_aug = bbs_aug.to_xyxy_array()
    bbs_rect = bbs_aug
    for bbs_aug_coord in bbs_aug:
      x_c = (bbs_aug_coord[0] + bbs_aug_coord[2]) / 2
      y_c = (bbs_aug_coord[1] + bbs_aug_coord[3]) / 2
      w = bbs_aug_coord[3] - bbs_aug_coord[1]
      h = bbs_aug_coord[2] - bbs_aug_coord[0]
      bbs_aug_coord = np.array([x_c, y_c, h, w])

    new_annotation = ""
    for box in bbs_aug:
      new_annotation += str(info_list[0])+' '+' '.join(map(str,translate_coord(image,box))) + "\n"
    
    
    try:
      image_aug = cv2.cvtColor(image_aug,cv2.COLOR_RGB2BGR)
      for rect in bbs_rect:
        image_aug = cv2.rectangle(image_aug, (rect[0], rect[1]), (rect[2], rect[3]), (0,0,255), 3)
      cv2.imwrite(path + "/images/train/aug_"+str(angle)+str(i)+".jpg",image_aug)

      # cv2.imwrite("/content/aug_"+str(angle)+str(i)+".jpg",image_aug)

      with open(path+"/labels/train/aug_"+str(angle)+str(i)+".txt",'w') as f:
        f.write(new_annotation)
      i+=1
    except:
      pass

In [15]:
import imgaug as ia
import imgaug.augmenters as iaa
import imageio
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import cv2
from google.colab.patches import cv2_imshow
from os import listdir
import numpy as np

ia.seed(1)
path = '/content/Yooninahong/2nd_data'
image_list = listdir(path+"/images/train/")
print(len(image_list))
for angle in [30, 60]:
  augmentation_angle(image_list,path,angle)

1431


In [18]:
len(listdir('/content/Yooninahong/2nd_data/labels/train'))

3753

In [2]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 10732, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10732 (delta 3), reused 6 (delta 1), pack-reused 10722
Receiving objects: 100% (10732/10732), 10.89 MiB | 26.50 MiB/s, done.
Resolving deltas: 100% (7411/7411), done.


In [3]:
!pip install -r /content/yolov5/requirements.txt

     |████████████████████████████████| 596 kB 7.8 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [10]:
from glob import glob

train_img_list = glob('/content/Yooninahong/dataset/images/train/*.jpg')
val_img_list = glob('/content/Yooninahong/dataset/images/val/*.jpg')

In [11]:
with open('/content/Yooninahong/dataset/train.txt','w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open('/content/Yooninahong/dataset/val.txt','w') as f:
  f.write('\n'.join(val_img_list) + '\n')

In [12]:
import yaml
import os
data = dict()
data['train'] = '/content/Yooninahong/dataset/train.txt'
data['val'] = '/content/Yooninahong/dataset/val.txt'
data['nc'] = '9'
data['names'] = ['electricscooter','brailleblock','person','bike','motorcycle_F','bollard_short','bollard_long','car','peron_b']

with open('/content/Yooninahong/dataset/data.yaml','w') as f:
  yaml.dump(data,f)
print(data)

{'train': '/content/Yooninahong/dataset/train.txt', 'val': '/content/Yooninahong/dataset/val.txt', 'nc': '9', 'names': ['electricscooter', 'brailleblock', 'person', 'bike', 'motorcycle_F', 'bollard_short', 'bollard_long', 'car', 'peron_b']}


In [13]:
with open('/content/Yooninahong/dataset/data.yaml', 'r') as f:
  data = yaml.load(f,Loader = yaml.Loader)
print(data)

{'names': ['electricscooter', 'brailleblock', 'person', 'bike', 'motorcycle_F', 'bollard_short', 'bollard_long', 'car', 'peron_b'], 'nc': '9', 'train': '/content/Yooninahong/dataset/train.txt', 'val': '/content/Yooninahong/dataset/val.txt'}


In [16]:
!python /content/yolov5/train.py --img 320 --batch 64 --epoch 100 --data /content/Yooninahong/2nd_data/data.yaml --cfg /content/yolov5/models/yolov5s.yaml --weights /content/yolov5/models/yolov5s.pt --name ynh_result

train: weights=/content/Yooninahong/best_s_pretrained_x.pt, cfg=/content/yolov5/models/yolov5m.yaml, data=/content/Yooninahong/dataset/data.yaml, hyp=yolov5/data/hyps/hyp.scratch.yaml, epochs=100, batch_size=128, imgsz=320, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=ynh_result, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-213-gd5966c9 torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=

In [17]:
from google.colab import files
files.download('/content/yolov5/runs/train/ynh_result3/weights/best.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
!python /content/yolov5/detect.py --weight /content/best_0126.pt --img 640 --conf 0.4 --source /content/4th.mp4

detect: weights=['/content/best_0126.pt'], source=/content/4th.mp4, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.3, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-216-g856d4e5 torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 290 layers, 20885262 parameters, 0 gradients, 48.0 GFLOPs
video 1/1 (1/111) /content/4th.mp4: 384x640 1 motorcycle_F, Done. (0.019s)
video 1/1 (2/111) /content/4th.mp4: 384x640 1 motorcycle_F, Done. (0.015s)
video 1/1 (3/111) /content/4th.mp4: 384x640 1 motorcycle_F, Done. (0.015s)
video 1/1 (4/111) /content/4th.mp4: 384x640 Done. (0.014s)
video 1/1 (5/111) /content/4th.mp4: 384x640 Done. (0.014s)
video 1/1 (6/111)